In [23]:
import numpy as np

In [34]:
# Define a class for a simple queue
class MyQUEUE:
    def __init__(self):
        self.holder = []

    def enqueue(self, val):
        self.holder.append(val)

    def dequeue(self):
        val = None
        try:
            val = self.holder[0]
            if len(self.holder) == 1:
                self.holder = []
            else:
                self.holder = self.holder[1:]
        except IndexError:
            pass
        return val
    def IsEmpty(self):
        return len(self.holder) == 0

In [35]:
# Function to find all possible paths in a graph using a queue
def chemins(graph, start, end, q):
    L = []
    q.enqueue([start])

    while not q.IsEmpty():
        tmp_path = q.dequeue()
        last_node = tmp_path[-1]

        if last_node == end:
            L.append(tmp_path)
        else:
            for link_node, value in enumerate(graph[last_node]):
                if value == 1 and link_node not in tmp_path:
                    new_path = tmp_path + [link_node]
                    q.enqueue(new_path)

    return L

In [36]:
# Function to update the state matrix based on certain conditions
def etatupd(M, i, j, seuil, S, AS, NTL):
    for x in range(len(M)):
        for y in range(len(M)):
            if M[x][y] == 1 and x != y:
                if AS[x][y] < seuil[x][y] or S[x][y] == 0:
                    if NTL[x][y] == 1:
                        S[x][y] = 0
                    else:
                        NTL[x][y] = NTL[x][y] - 1
                        S[x][y] = 1
    return [S, NTL]


In [37]:
# Function to calculate a specific value P based on given parameters
def calculateP(M, i, j, NV, lg, AS, NTL, S, a, b, c, d):
    n = 0
    [S, NTL] = etatupd(M, i, j, seuil, S, AS, NTL)

    for k in range(len(M)):
        if M[i][k] == 1:
            denominator = (((NV[i][k]) ** a) * ((lg[i][k]) ** c))
            if denominator != 0:
                n += (((AS[i][k]) ** b) * ((NTL[i][k]) ** d) * S[i][k]) / denominator

    try:
        denominator = (((NV[i][j]) ** a) * ((lg[i][j]) ** c))
        if denominator != 0:
            P = ((AS[i][j] ** b) * ((NTL[i][j]) ** d) * (S[i][j])) / denominator
            return P
        else:
            return 0
    except ZeroDivisionError:
        return 0

In [38]:
# Function to count the number of successors for a node
def nomsucc(i, M):
    nomsucc = 0
    for j in range(len(M)):
        if M[i][j] != 0:
            nomsucc += 1
    return nomsucc

In [39]:
# Function to find the successor for a given node in a path
def succ(i, k):
    for j in range(len(k)):
        if k[j] == i:
            a = k[j + 1]
            return a

In [40]:
# Function to calculate the optimal path and its maximum weight
def calculate_optimal_path(paths, start_node, end_node, graph, NV, lg, AS, NTL, S, a, b, c, d):
    optimal_path = None
    max_weight = 0

    for k in paths:
        X = start_node
        W = 0  # Initialize W for each path

        while succ(X, k) != end_node:
            if nomsucc(succ(X, k), graph) >= 2:
                Y = succ(X, k)
                P = calculateP(graph, X, Y, NV, lg, AS, NTL, S, a, b, c, d)
                if P == 0:
                    break
                else:
                    X = succ(X, k)
                    W += P
            else:
                Z = X
                while nomsucc(succ(X, k), graph) == 1 and succ(X, k) != end_node:
                    X = succ(X, k)
                    P = calculateP(graph, Z, X, NV, lg, AS, NTL, S, a, b, c, d)
                    if P == 0:
                        break
                    if succ(X, k) != end_node:
                        P = 1
                    else:
                        P = calculateP(graph, Z, X, NV, lg, AS, NTL, S, a, b, c, d)
                        if P == 0:
                            break
                    W += P

        if W > max_weight:
            max_weight = W
            optimal_path = k

    return optimal_path, max_weight

In [41]:
# Example graph represented as an adjacency matrix
graph = [
    [0, 1, 1, 0],
    [0, 0, 1, 1],
    [0, 0, 0, 1],
    [0, 0, 0, 0]
]

# Parameters
start_node = 0
end_node = 3
seuil = [
    [0, 1, 2, 0],
    [0, 0, 3, 4],
    [0, 0, 0, 5],
    [0, 0, 0, 0]
]
NV = [
    [0, 2, 3, 0],
    [0, 0, 4, 5],
    [0, 0, 0, 6],
    [0, 0, 0, 0]
]
lg = [
    [0, 1, 2, 0],
    [0, 0, 3, 4],
    [0, 0, 0, 5],
    [0, 0, 0, 0]
]
AS = [
    [0, 1, 1, 0],
    [0, 0, 1, 1],
    [0, 0, 0, 1],
    [0, 0, 0, 0]
]
NTL = [
    [0, 1, 2, 0],
    [0, 0, 3, 4],
    [0, 0, 0, 5],
    [0, 0, 0, 0]
]

# Initialize S matrix
S = np.ones_like(AS)

# Create a MyQUEUE instance
path_queue = MyQUEUE()

# Find paths
paths = chemins(graph, start_node, end_node, path_queue)
print("All paths:", paths)

# Example usage
optimal_path, max_weight = calculate_optimal_path(paths, start_node, end_node, graph, NV, lg, AS, NTL, S, a, b, c, d)

# Print the optimal path
if optimal_path:
    print("Le chemin Optimal est le suivant: ")
    print(optimal_path)
    print("Maximum Weight:", max_weight)

All paths: [[0, 1, 3], [0, 2, 3], [0, 1, 2, 3]]
Le chemin Optimal est le suivant: 
[0, 1, 3]
Maximum Weight: 0.5
